<a href="https://colab.research.google.com/github/jayrom/enterprise_challenge_reply/blob/main/sprint_4/src/predict_data_generation_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulação de dados de sensores industriais
*Dataset de manutenção (rotulado)*

In [2]:
import pandas as pd
import numpy as np
import random

def gerar_dados_teste(n_registros=500, inicio_falha_percent=0.65, random_seed=123):
    """
    Gera um conjunto de dados de teste simulando uma única linha do tempo de um equipamento
    que começa normal e desenvolve um modo de falha escolhido aleatoriamente.

    Salva dois arquivos: um com rótulos (gabarito) e outro sem (para predição).
    """
    print("Iniciando a geração do conjunto de teste...")

    # Semente aleatória diferente da de treinamento para garantir dados inéditos
    np.random.seed(random_seed)

    # --- Configurações Iniciais ---
    intervalo_minutos = 10
    start_time = pd.Timestamp('2025-09-27 08:00:00') # Data de início para o teste
    timestamps = pd.to_datetime([start_time + pd.Timedelta(minutes=i*intervalo_minutos) for i in range(n_registros)])

    # Escolhe aleatoriamente o motor e o modo de falha para este teste
    modos_falha_possiveis = ['desgaste_rolamento', 'desbalanceamento']
    modo_falha_simulado = random.choice(modos_falha_possiveis)
    device_id = 'motor_teste_C'

    print(f"Simulando uma falha do tipo: '{modo_falha_simulado}' para o device '{device_id}'")

    # --- Geração dos Dados Base (Estado Normal) ---
    temperature = np.random.normal(31, 2, n_registros)
    current = np.random.normal(1.6, 0.1, n_registros)
    # Simulação dos 3 eixos de vibração com gravidade em Z
    vib_x = np.random.normal(0, 0.05, n_registros)
    vib_y = np.random.normal(0, 0.05, n_registros)
    vib_z = np.random.normal(9.8, 0.05, n_registros)

    # --- Simulação da Falha Progressiva ---
    start_failure_idx = int(n_registros * inicio_falha_percent)
    days_to_failure = np.zeros(n_registros) # Inicializa com 0 (ou outro valor p/ normal)
    failure_mode = ['normal'] * n_registros

    for i in range(start_failure_idx, n_registros):
        progress = (i - start_failure_idx) / (n_registros - start_failure_idx)

        # Define os rótulos para o período de falha
        failure_mode[i] = modo_falha_simulado
        days_remaining = (n_registros - 1 - i) * intervalo_minutos / (60 * 24)
        days_to_failure[i] = max(0, days_remaining)

        # Aplica a anomalia correspondente ao modo de falha
        if modo_falha_simulado == 'desgaste_rolamento':
            # Padrão senoidal
            temperature[i] += 15 * np.sin(progress * np.pi) + np.random.normal(0, 1)
            current[i] += 5 * np.sin(progress * np.pi) + np.random.normal(0, 0.5)
            vib_x[i] += 1.0 * np.sin(progress * np.pi) + np.random.normal(0, 0.1)
            vib_y[i] += 1.2 * np.sin(progress * np.pi) + np.random.normal(0, 0.1)
            vib_z[i] += 0.3 * np.sin(progress * np.pi) + np.random.normal(0, 0.05)

        elif modo_falha_simulado == 'desbalanceamento':
            # Padrão exponencial
            if progress > 0.3: # Delay
                temperature[i] += 10 * np.sqrt(progress - 0.3) + np.random.normal(0, 1.2)
                current[i] += 4 * np.sqrt(progress - 0.3) + np.random.normal(0, 0.6)
            vib_x[i] += 0.5 * np.exp(progress * 1.5) + np.random.normal(0, 0.10)
            vib_y[i] += 2.5 * np.exp(progress * 1.5) + np.random.normal(0, 0.20)
            vib_z[i] += 0.2 * np.exp(progress * 1.5) + np.random.normal(0, 0.08)

    # --- Finalização e Criação dos DataFrames ---

    # Calcular a magnitude vetorial (a feature final de vibração)
    vibration_magnitude = np.sqrt(vib_x**2 + vib_y**2 + vib_z**2)

    # DataFrame completo (com gabarito)
    df_final_com_rotulos = pd.DataFrame({
        'timestamp': timestamps,
        'device_id': device_id,
        'temperature_c': temperature.round(2),
        'current_amps': current.round(2),
        'vibration_magnitude_mss': vibration_magnitude.round(2),
        'days_to_failure': days_to_failure.round(2),
        'failure_mode': failure_mode
    })

    # DataFrame para predição (sem gabarito)
    df_para_predicao = df_final_com_rotulos.drop(columns=['days_to_failure', 'failure_mode'])

    # --- Salvando os arquivos ---
    path_com_rotulos = 'dados_teste_com_rotulos.csv'
    path_para_predicao = 'dados_teste_para_predicao.csv'

    df_final_com_rotulos.to_csv(path_com_rotulos, index=False)
    df_para_predicao.to_csv(path_para_predicao, index=False)

    print("\nConjunto de teste gerado com sucesso!")
    print(f" -> Arquivo com rótulos (gabarito) salvo como: '{path_com_rotulos}'")
    print(f" -> Arquivo sem rótulos (para predição) salvo como: '{path_para_predicao}'")

    print("\nAmostra dos dados gerados (com rótulos):")
    print(df_final_com_rotulos.head(3))
    print("...")
    print(df_final_com_rotulos.tail(3))

# Executar a função para gerar os dados
if __name__ == '__main__':
    gerar_dados_teste()

Iniciando a geração do conjunto de teste...
Simulando uma falha do tipo: 'desgaste_rolamento' para o device 'motor_teste_C'

Conjunto de teste gerado com sucesso!
 -> Arquivo com rótulos (gabarito) salvo como: 'dados_teste_com_rotulos.csv'
 -> Arquivo sem rótulos (para predição) salvo como: 'dados_teste_para_predicao.csv'

Amostra dos dados gerados (com rótulos):
            timestamp      device_id  temperature_c  current_amps  \
0 2025-09-27 08:00:00  motor_teste_C          28.83          1.68   
1 2025-09-27 08:10:00  motor_teste_C          32.99          1.61   
2 2025-09-27 08:20:00  motor_teste_C          31.57          1.57   

   vibration_magnitude_mss  days_to_failure failure_mode  
0                     9.71              0.0       normal  
1                     9.74              0.0       normal  
2                     9.85              0.0       normal  
...
              timestamp      device_id  temperature_c  current_amps  \
497 2025-09-30 18:50:00  motor_teste_C        

### Importar módulos

### Configurações iniciais da simulação

### Parâmetros de dados normais (média e desvio padrão para ruído)

### Parâmetros de falha (taxa de aumento por dia)

### Geração dos dados